In [10]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from sklearn.preprocessing import LabelEncoder

In [22]:
# 데이터 불러오기

base_path = '../../../Machine-Running_LJW/Group Project/'

submission_path = base_path + 'sample_submission_V2.csv' 
train_path = base_path + 'train_V2.csv'
test_path = base_path + 'test_V2.csv'

submission = pd.read_csv(submission_path)
train= pd.read_csv(train_path)
test=pd.read_csv(test_path)

In [ ]:
# # Kaggle 제출 시

# # 데이터 불러오기

# submission_path = '/kaggle/input/pubg-finish-placement-prediction/sample_submission_V2.csv' 
# train_path = '/kaggle/input/pubg-finish-placement-prediction/train_V2.csv'
# test_path = '/kaggle/input/pubg-finish-placement-prediction/test_V2.csv'

# submission = pd.read_csv(submission_path)
# train= pd.read_csv(train_path)
# test=pd.read_csv(test_path)

In [23]:
# train data는 결측치 제거 후 copy, test data는 copy

train = train.dropna()
train01 = train.copy()

test01 = test.copy()


In [24]:
#데이터 타입 최적화-> 메모리 최적화
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    #start_mem = df.memory_usage().sum() / 1024**2
    #print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    #end_mem = df.memory_usage().sum() / 1024**2
    #print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    #print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df


In [25]:
train01 = reduce_mem_usage(train01)
test01 = reduce_mem_usage(test01)

In [26]:
#함수로 만들자
def feature_engineering(df):
    #수치형 데이터
    df_col= [ 'assists', 'boosts', 'damageDealt', 'DBNOs',
       'headshotKills', 'heals', 'killPlace', 'killPoints', 'kills','matchType',
       'killStreaks', 'longestKill', 'matchDuration', 'maxPlace',
       'numGroups', 'rankPoints', 'revives', 'rideDistance', 'roadKills',
       'swimDistance', 'teamKills', 'vehicleDestroys', 'walkDistance',
       'weaponsAcquired', 'winPoints']
    # df_t는 데이터를 그룹아이디 기준으로 평균을 내기 위해 저장할 데이터프레임
    df_t=df.copy()
    #라벨인코딩
    le = LabelEncoder()
    df_t['matchType']= le.fit_transform(df_t['matchType'])
    df_t['matchType']
    # 그룹아이디를 기준으로 수치형 컬럼들을 평균화
    df_t= df_t.groupby(['groupId'])[df_col].agg('mean').reset_index()
    
    return df_t


In [27]:
# df에는 앞에서 만든 함수를 통해 피처 엔지니어링 수치형 데이터를 평균을 냄
train02 = feature_engineering(train01)
test02 = feature_engineering(test01)

In [28]:
#원본 train01 데이터의 순서와 로우를 잃지 않기 위해 그룹아이디와 그에 맞는 타겟벨류를 가지고옴
train03=train01.copy()
train03=train03[['groupId','winPlacePerc']]
# 데이터 합치기- 앞에서 데이터를 피처 엔지니어링한 train02를 원본 데이터인 train03의 순서에 맞게 넣기 위해 merge를 사용.
# how='left'는 두 데이터프레임중 기준을 잡을 데이터프레임을 정함.
# 원본train03 의 순서와 로우수를 맞춰줌
train03=pd.merge(train03,train02,how='left',on='groupId')
# 모델 학습을 위해 두가지 변수를 제거 killplace는 큰 비중을 차지하여 공부를 위해 제거 groupId는 학습을 위해 수치형 데이터만
#남게 하기 위해 제거 
train03=train03.drop(columns=['killPlace','groupId'])
train03
    

,winPlacePerc,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPoints,kills,matchType,...,rankPoints,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints
0,0.444336,0.00,0.000000,102.187500,0.5,0.25,0.000000,1250.5,1.00,15.0,...,-1.000000,0.00,0.00,0.0,0.000000,0.0,0.0,183.0000,1.250000,1476.25
1,0.640137,0.50,1.500000,142.750000,0.5,0.00,3.250000,0.0,1.00,15.0,...,1482.750000,0.00,3196.00,0.0,2.759766,0.0,0.0,1893.0000,6.000000,0.00
2,0.775391,0.50,1.500000,107.312500,0.5,0.00,1.000000,0.0,1.00,2.0,...,1492.500000,0.00,170.25,0.0,0.000000,0.0,0.0,640.5000,2.000000,0.00
3,0.166748,0.00,0.000000,32.906250,0.0,0.00,0.000000,0.0,0.00,15.0,...,1408.000000,0.00,0.00,0.0,0.000000,0.0,0.0,202.7500,3.000000,0.00
4,0.187500,0.00,0.000000,100.000000,0.0,0.00,0.000000,0.0,1.00,13.0,...,1560.000000,0.00,0.00,0.0,0.000000,0.0,0.0,49.7500,2.000000,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4446960,0.178589,0.00,0.400000,119.500000,1.0,0.20,0.400000,1158.8,1.00,15.0,...,-1.000000,0.20,1150.00,0.0,0.000000,0.0,0.0,697.0000,3.800000,1515.20
4446961,0.293457,0.00,1.000000,44.156250,0.0,0.00,0.000000,0.0,0.00,12.0,...,1501.000000,0.00,0.00,0.0,0.000000,0.0,0.0,81.6875,6.000000,0.00
4446962,0.481445,0.00,0.333333,28.359375,0.0,0.00,0.666667,0.0,0.00,15.0,...,1346.666667,0.00,0.00,0.0,7.769531,0.0,0.0,613.0000,2.666667,0.00
4446963,0.799805,0.25,2.250000,77.375000,0.5,0.25,1.000000,0.0,1.25,15.0,...,1366.500000,0.75,141.50,0.0,0.000000,0.0,0.0,2576.0000,6.000000,0.00


In [29]:
#원본 데이터의 순서와 로우 수를 위해 그룹아이디를 가지고옴
test03=test01.copy()
test03=test03[['groupId']]
#데이터 합치기- 위와 동일
test03=pd.merge(test03,test02,how='left',on='groupId')
test03=test03.drop(columns=['killPlace','groupId'])
test03

,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPoints,kills,matchType,killStreaks,...,rankPoints,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints
0,0.250000,0.000000,31.40625,0.250000,0.0,0.000000,0.000000,0.000000,15.0,0.000000,...,1446.250000,0.000000,673.5,0.0,0.000000,0.000000,0.0,609.000,2.500000,0.000000
1,0.500000,5.000000,388.25000,2.000000,0.5,3.500000,0.000000,3.500000,3.0,1.500000,...,1511.000000,1.000000,4672.0,0.0,0.000000,0.000000,0.0,2246.000,6.500000,0.000000
2,0.750000,2.250000,372.50000,2.000000,1.0,3.250000,0.000000,3.000000,15.0,0.750000,...,1542.500000,0.000000,643.0,0.0,0.000000,0.000000,0.0,1100.000,3.750000,0.000000
3,0.000000,0.000000,82.75000,0.500000,0.0,0.000000,0.000000,0.000000,3.0,0.000000,...,1486.500000,0.000000,0.0,0.0,0.000000,0.000000,0.0,1964.000,4.000000,0.000000
4,0.333333,2.333333,220.62500,1.333333,1.0,1.666667,0.000000,2.333333,15.0,1.000000,...,1502.333333,0.333333,0.0,0.0,0.000000,0.000000,0.0,2898.000,6.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1934169,0.333333,2.000000,226.00000,2.000000,0.0,6.333333,1311.666667,2.333333,15.0,1.333333,...,-1.000000,0.666667,0.0,0.0,0.000000,0.000000,0.0,1896.000,5.000000,1517.666667
1934170,0.000000,0.000000,35.03125,0.000000,0.0,0.250000,1435.750000,0.000000,15.0,0.000000,...,-1.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,1368.000,4.000000,1523.500000
1934171,0.333333,1.000000,91.75000,0.666667,0.0,4.000000,0.000000,1.333333,15.0,1.333333,...,1485.000000,0.666667,0.0,0.0,2.599609,0.333333,0.0,3028.000,3.333333,0.000000
1934172,0.500000,3.000000,186.62500,1.000000,0.5,11.000000,0.000000,2.000000,3.0,1.500000,...,1456.000000,0.500000,0.0,0.0,0.000000,0.000000,0.0,1905.000,4.500000,0.000000


In [30]:
#모델링을 위해 winplacePerc를 제거하고, y로 값을 따로 만듬
x=train03.drop(columns='winPlacePerc')
y=train03[['winPlacePerc']]

In [31]:
#스케일링, test03도 스케일링을 함께 함
scaler = StandardScaler()
test03=scaler.fit_transform(test03)
x=scaler.fit_transform(x)
x_train,x_val,y_train,y_val = train_test_split(x,y,test_size=0.2,random_state=0)
x_train.shape,x_val.shape,y_train.shape,y_val.shape

((3557572, 24), (889393, 24), (3557572, 1), (889393, 1))

In [32]:
# lightgbm 활용-> 학습용
import lightgbm as lgb
d_train = lgb.Dataset(x_train, label=y_train)
d_val = lgb.Dataset(x_val,label=y_val)
params = {"objective" : "regression", "metric" : "mae", 'n_estimators':1000, 'early_stopping_rounds':200,
              "num_leaves" : 31, "learning_rate" : 0.05, "bagging_fraction" : 0.7,
               "bagging_seed" : 0, "num_threads" : 4,"colsample_bytree" : 0.7
             }

clf = lgb.train(params, d_train, valid_sets=[d_train, d_val],early_stopping_rounds=200, verbose_eval=1000)
print(clf)

C:\Users\brown\miniforge3\envs\csw\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
C:\Users\brown\miniforge3\envs\csw\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\brown\miniforge3\envs\csw\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.290072 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4198
[LightGBM] [Info] Number of data points in the train set: 3557572, number of used features: 24
[LightGBM] [Info] Start training from score 0.472922
Training until validation scores don't improve for 200 rounds
[1000]	training's l1: 0.0669571	valid_1's l1: 0.067139
Did not meet early stopping. Best iteration is:
[1000]	training's l1: 0.0669571	valid_1's l1: 0.067139


In [13]:
# 학습한 모델 clf로 테스트파일을 예측함 pred는 테스트 파일의 y벨류가 들어있음
pred = clf.predict(test03, num_iteration=clf.best_iteration)

In [14]:
# Kaggle 제출 시 사용

# 제출을 위해 파일 변환
df1 = pd.read_csv('sample_submission_V2.csv')
df1['winPlacePerc']=pred
df1.to_csv('submission.csv',index = False)